### Authenticate with Google Generative AI

- Please visit [Google AI Studio](https://ai.google.dev/) to generate API key.
- Export the key to your environment as `GEMINI_API_KEY`

In [3]:
import os
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")

In [7]:
import google.generativeai as genai

generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash', generation_config=generation_config)

In [20]:
def upload_file(file_path: str, file_type: str = None):
    """Uploads the given file to Gemini

    Refer: https://ai.google.dev/gemini-api/docs/prompting_with_media
    """
    file = genai.upload_file(file_path, mime_type=file_type)
    print(f"{file} uploaded successfully")
    return file

In [ ]:
def get_file_status(file):
    file = genai.get_file(file.name)
    return file.state.name

def check_file_is_active(file):
    while get_file_status(file=file) == "PROCESSING":
        print(".", end="", flush=True)
    if get_file_status(file=file) != "ACTIVE":
        raise Exception(f"{file} failed to process")
    print("")
    return True

def check_all_files_status(files):
    print("Processing files...", end="", flush=True)
    status = True
    for file in files:
        status = status and check_file_is_active(file=file)

    print("All files processed" if status else "Experiencing difficulties in processing files")

In [35]:
genai.list_models

<function google.generativeai.models.list_models(*, page_size: 'int | None' = 50, client: 'glm.ModelServiceClient | None' = None, request_options: 'helper_types.RequestOptionsType | None' = None) -> 'model_types.ModelsIterable'>

In [36]:
file_paths = ["/home/naveen/Dowloads/Deadpool dance Bye-bye-bye.mp4"]

files = [
    upload_file(file_path=file) for file in file_paths
]
check_all_files_status(files=files)

genai.File({
    'name': 'files/y2eqfsq55njl',
    'display_name': 'Deadpool dance Bye-bye-bye.mp4',
    'mime_type': 'video/mp4',
    'sha256_hash': 'YTY5YzMzZDhmOGMxMzc4ZjMxODhjMTZlMTMzYTI4NDkzZDczODI1YzQ2OWY0NGQyMzRmMmJlMjg1ZjQ5YzAwNg==',
    'size_bytes': '9500941',
    'state': 'PROCESSING',
    'uri': 'https://generativelanguage.googleapis.com/v1beta/files/y2eqfsq55njl',
    'create_time': '2024-11-07T05:40:31.046078Z',
    'expiration_time': '2024-11-09T05:40:30.984894022Z',
    'update_time': '2024-11-07T05:40:31.046078Z'}) uploaded successfully
Processing files.....All files processed


In [40]:
chat_session = model.start_chat(
    history=[{
        "role": "user",
        "parts": [files[0]]
    }]
)

In [41]:
chat_session

ChatSession(
    model=genai.GenerativeModel(
        model_name='models/gemini-1.5-flash',
        generation_config={'temperature': 1, 'top_p': 0.95, 'top_k': 64, 'max_output_tokens': 8192, 'response_mime_type': 'text/plain'},
        safety_settings={},
        tools=None,
        system_instruction=None,
        cached_content=None
    ),
    history=[protos.Content({'parts': [{'file_data': {'file_uri': 'https://gene.../y2eqfsq55njl', 'mime_type': 'video/mp4'}}], 'role': 'user'})]
)

In [46]:
chat_session.history

[parts {
   file_data {
     mime_type: "video/mp4"
     file_uri: "https://generativelanguage.googleapis.com/v1beta/files/y2eqfsq55njl"
   }
 }
 role: "user"]